In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
XGB_WEIGHT = 0.6415
BASELINE_WEIGHT = 0.056
OLS_WEIGHT = 0.0828

XGB1_WEIGHT = 0.8083

BASELINE_PRED = 0.0115

In [3]:
prop = pd.read_csv('properties_2016.csv')
train = pd.read_csv('train_2016_v2.csv')

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


LightGBM

In [4]:
for c,dtype in zip(prop.columns,prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [7]:
df_train= train.merge(prop,how='left',on='parcelid')
df_train.fillna(df_train.median(),inplace=True)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,7.0,616.0,2.0,3.0,4.0,4.0,...,1.0,1,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,14.0,6.037107e+13
1,14366692,-0.1684,2016-01-01,1.0,7.0,616.0,3.5,4.0,4.0,7.0,...,1.0,1,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,14.0,6.037615e+13
2,12098116,-0.0040,2016-01-01,1.0,7.0,616.0,3.0,2.0,4.0,4.0,...,1.0,1,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,14.0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,7.0,616.0,2.0,2.0,4.0,4.0,...,1.0,1,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,14.0,6.037296e+13
4,14432541,-0.0050,2016-01-02,1.0,7.0,616.0,2.5,4.0,4.0,7.0,...,2.0,1,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,14.0,6.059042e+13
5,11509835,-0.2705,2016-01-02,1.0,7.0,616.0,4.0,4.0,4.0,1.0,...,1.0,1,880650.0,2447951.0,2015.0,1567301.0,27126.570312,NaN,14.0,6.037621e+13
6,12286022,0.0440,2016-01-02,1.0,7.0,616.0,1.0,2.0,4.0,7.0,...,1.0,1,64549.0,111521.0,2015.0,46972.0,2304.969971,NaN,14.0,6.037542e+13
7,17177301,0.1638,2016-01-02,1.0,7.0,616.0,2.5,3.0,4.0,7.0,...,2.0,1,107000.0,306000.0,2015.0,199000.0,3745.500000,NaN,14.0,6.111003e+13
8,14739064,-0.0030,2016-01-02,1.0,7.0,616.0,1.0,2.0,4.0,7.0,...,1.0,1,66834.0,210064.0,2015.0,143230.0,2172.879883,NaN,14.0,6.059042e+13
9,14677559,0.0843,2016-01-03,1.0,7.0,616.0,2.0,2.0,4.0,7.0,...,1.0,1,109977.0,190960.0,2015.0,80983.0,1940.260010,NaN,14.0,6.059063e+13


In [9]:
x_train = df_train.drop(['parcelid','logerror','transactiondate','propertyzoningdesc',
                        'propertycountylandusecode','fireplacecnt','fireplaceflag'],axis=1)
y_train = df_train['logerror'].values
print(x_train.shape,y_train.shape)

(90275, 53) (90275,)


In [10]:
train_columns = x_train.columns

In [20]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

In [23]:
x_train = x_train.values.astype(np.float32,copy=False)

In [26]:
d_train = lgb.Dataset(x_train,label=y_train)

In [31]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'
params['sub_feature'] = 0.345
params['bagging_fraction'] = 0.85
params['bagging_freq'] = 40
params['num_leaves'] = 512
params['min_data'] = 500
params['min_hessian'] = 0.05
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

In [32]:
clf = lgb.train(params,d_train,430)

In [34]:
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop,on='parcelid',how='left')

In [36]:
x_test = df_test[train_columns]

In [37]:
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
x_test = x_test.values.astype(np.float32,copy=False)

In [40]:
p_test = clf.predict(x_test)

xgboost

In [44]:
properties = pd.read_csv('properties_2016.csv')

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
for c in properties.columns:
    properties[c] = properties[c].fillna(-1)
    if properties[c].dtype =='object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

In [46]:
train_df = train.merge(properties,how='left',on='parcelid')

In [47]:
x_train = train_df.drop(['parcelid','logerror','transactiondate'],axis=1)
x_test = properties.drop(['parcelid'],axis=1)

In [50]:
train_df = train_df[train_df.logerror > -0.4]
train_df = train_df[train_df.logerror < 0.419]
x_train = train_df.drop(['parcelid','logerror','transactiondate'],axis=1)
y_train = train_df['logerror'].values.astype(np.float32)
y_mean = np.mean(y_train)

In [51]:
x_train.shape

(88528, 57)

In [52]:
x_test.shape

(2985217, 57)

In [54]:
xgb_params = {}
xgb_params['eta'] = 0.037
xgb_params['max_depth'] = 5
xgb_params['subsample'] = 0.80
xgb_params['objective'] = 'reg:linear'
xgb_params['eval_metric'] = 'mae'
xgb_params['lambda'] = 0.8
xgb_params['alpha'] = 0.4
xgb_params['base_score'] = y_mean
xgb_params['silent'] = 1

In [57]:
dtrain = xgb.DMatrix(x_train,y_train)
dtest = xgb.DMatrix(x_test)

In [63]:
model = xgb.train(xgb_params,dtrain,num_boost_round=250)

In [64]:
xgb_pred1 = model.predict(dtest)

In [67]:
pd.DataFrame(xgb_pred1).head()

,0
0,-0.042947
1,-0.029738
2,0.027966
3,0.069254
4,0.014018


run xgboost again

In [68]:
xgb_params_v2 = {}
xgb_params_v2['eta'] = 0.033
xgb_params_v2['max_depth'] = 6
xgb_params_v2['subsample'] = 0.8
xgb_params_v2['objective'] = 'reg:linear'
xgb_params_v2['eval_metric'] = 'mae'
xgb_params_v2['base_score'] = y_mean
xgb_params_v2['silent'] = 1

In [70]:
model_v2 = xgb.train(xgb_params_v2,dtrain,num_boost_round=150)

In [71]:
xgb_pred2 = model_v2.predict(dtest)

In [72]:
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2

In [74]:
pd.DataFrame(xgb_pred).head()

,0
0,-0.044033
1,-0.028420
2,0.025723
3,0.066739
4,0.012370


OLS

In [75]:
np.random.seed(17)
random.seed(17)

In [78]:
train = pd.read_csv('train_2016_v2.csv',parse_dates=['transactiondate'])

In [81]:
properties = pd.read_csv('properties_2016.csv')
submission = pd.read_csv('sample_submission.csv')

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [85]:
def get_features(df):
    df['transactiondate'] = pd.to_datetime(df['transactiondate'])
    df['transactiondate_year'] = df['transactiondate'].dt.year
    df['transactiondate_month'] = df['transactiondate'].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

In [86]:
def MAE(y,ypred):
    return np.sum([abs(y[i] - ypred[i]) for i in range(len(y))])/len(y)

In [87]:
train = pd.merge(train,properties,how='left',on='parcelid')
y = train['logerror'].values
test = pd.merge(submission,properties,how='left',left_on='ParcelId',right_on='parcelid')
properties = []

In [89]:
exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] =='O']+['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

In [92]:
train = get_features(train[col])
test['transactiondate'] = '2016-01-01'
test = get_features(test[col])

d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\users\zhangzerong\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [96]:
reg = LinearRegression(n_jobs=-1)
reg.fit(train,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [97]:
train = []
y =[]

In [98]:
test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']

In [101]:
lgb_weight = (1 - XGB_WEIGHT - BASELINE_WEIGHT)/(1-OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT/(1 - OLS_WEIGHT)
baseline_weight0 = BASELINE_WEIGHT/(1-OLS_WEIGHT)

In [103]:
pred0 = xgb_weight0*xgb_pred + baseline_weight0*BASELINE_PRED + lgb_weight*p_test

In [106]:
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0
    submission[test_columns[i]] = [float(format(x,'.4f')) for x in pred]
    print('predict...',i)

predict... 0
predict... 1
predict... 2
predict... 3
predict... 4
predict... 5


In [107]:
submission

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.0205,-0.0205,-0.0205,-0.0205,-0.0205,-0.0205
1,10759547,-0.0108,-0.0108,-0.0108,-0.0108,-0.0108,-0.0108
2,10843547,0.0809,0.0808,0.0808,0.0809,0.0808,0.0808
3,10859147,0.0554,0.0554,0.0553,0.0554,0.0554,0.0553
4,10879947,0.0181,0.0181,0.0181,0.0181,0.0181,0.0181
5,10898347,0.0236,0.0236,0.0236,0.0236,0.0236,0.0236
6,10933547,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107
7,10940747,0.0169,0.0168,0.0168,0.0169,0.0168,0.0168
8,10954547,-0.0253,-0.0253,-0.0253,-0.0253,-0.0253,-0.0253
9,10976347,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192
